# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fa9a0b22df0>
├── 'a' --> tensor([[ 0.2925,  0.9545, -0.6153],
│                   [-0.1786,  1.7654, -1.2716]])
└── 'x' --> <FastTreeValue 0x7fa9a0b22bb0>
    └── 'c' --> tensor([[ 0.7716,  0.7006, -1.4428,  0.9357],
                        [-0.4567, -0.1508,  1.0762,  0.4959],
                        [-1.8219, -0.5449, -1.3142,  0.1862]])

In [4]:
t.a

tensor([[ 0.2925,  0.9545, -0.6153],
        [-0.1786,  1.7654, -1.2716]])

In [5]:
%timeit t.a

63.9 ns ± 0.077 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fa9a0b22df0>
├── 'a' --> tensor([[ 1.1791, -0.8716, -0.9033],
│                   [ 0.5138, -0.7232, -0.1154]])
└── 'x' --> <FastTreeValue 0x7fa9a0b22bb0>
    └── 'c' --> tensor([[ 0.7716,  0.7006, -1.4428,  0.9357],
                        [-0.4567, -0.1508,  1.0762,  0.4959],
                        [-1.8219, -0.5449, -1.3142,  0.1862]])

In [7]:
%timeit t.a = new_value

64.4 ns ± 0.0492 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.2925,  0.9545, -0.6153],
               [-0.1786,  1.7654, -1.2716]]),
    x: Batch(
           c: tensor([[ 0.7716,  0.7006, -1.4428,  0.9357],
                      [-0.4567, -0.1508,  1.0762,  0.4959],
                      [-1.8219, -0.5449, -1.3142,  0.1862]]),
       ),
)

In [10]:
b.a

tensor([[ 0.2925,  0.9545, -0.6153],
        [-0.1786,  1.7654, -1.2716]])

In [11]:
%timeit b.a

51.9 ns ± 0.0426 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.0020, -0.0693, -1.1256],
               [-0.6185, -0.7134, -0.6490]]),
    x: Batch(
           c: tensor([[ 0.7716,  0.7006, -1.4428,  0.9357],
                      [-0.4567, -0.1508,  1.0762,  0.4959],
                      [-1.8219, -0.5449, -1.3142,  0.1862]]),
       ),
)

In [13]:
%timeit b.a = new_value

481 ns ± 0.304 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

971 ns ± 12.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 13.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 453 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 426 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fa977fd2940>
├── 'a' --> tensor([[[ 0.2925,  0.9545, -0.6153],
│                    [-0.1786,  1.7654, -1.2716]],
│           
│                   [[ 0.2925,  0.9545, -0.6153],
│                    [-0.1786,  1.7654, -1.2716]],
│           
│                   [[ 0.2925,  0.9545, -0.6153],
│                    [-0.1786,  1.7654, -1.2716]],
│           
│                   [[ 0.2925,  0.9545, -0.6153],
│                    [-0.1786,  1.7654, -1.2716]],
│           
│                   [[ 0.2925,  0.9545, -0.6153],
│                    [-0.1786,  1.7654, -1.2716]],
│           
│                   [[ 0.2925,  0.9545, -0.6153],
│                    [-0.1786,  1.7654, -1.2716]],
│           
│                   [[ 0.2925,  0.9545, -0.6153],
│                    [-0.1786,  1.7654, -1.2716]],
│           
│                   [[ 0.2925,  0.9545, -0.6153],
│                    [-0.1786,  1.7654, -1.2716]]])
└── 'x' --> <FastTreeValue 0x7fa977fd2880>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.1 µs ± 228 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fa9a0b22d90>
├── 'a' --> tensor([[ 0.2925,  0.9545, -0.6153],
│                   [-0.1786,  1.7654, -1.2716],
│                   [ 0.2925,  0.9545, -0.6153],
│                   [-0.1786,  1.7654, -1.2716],
│                   [ 0.2925,  0.9545, -0.6153],
│                   [-0.1786,  1.7654, -1.2716],
│                   [ 0.2925,  0.9545, -0.6153],
│                   [-0.1786,  1.7654, -1.2716],
│                   [ 0.2925,  0.9545, -0.6153],
│                   [-0.1786,  1.7654, -1.2716],
│                   [ 0.2925,  0.9545, -0.6153],
│                   [-0.1786,  1.7654, -1.2716],
│                   [ 0.2925,  0.9545, -0.6153],
│                   [-0.1786,  1.7654, -1.2716],
│                   [ 0.2925,  0.9545, -0.6153],
│                   [-0.1786,  1.7654, -1.2716]])
└── 'x' --> <FastTreeValue 0x7fa8f37a8f40>
    └── 'c' --> tensor([[ 0.7716,  0.7006, -1.4428,  0.9357],
                        [-0.4567, -0.1508,  1.0762,  0.4959],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 58.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.2 µs ± 38.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.2925,  0.9545, -0.6153],
                [-0.1786,  1.7654, -1.2716]],
       
               [[ 0.2925,  0.9545, -0.6153],
                [-0.1786,  1.7654, -1.2716]],
       
               [[ 0.2925,  0.9545, -0.6153],
                [-0.1786,  1.7654, -1.2716]],
       
               [[ 0.2925,  0.9545, -0.6153],
                [-0.1786,  1.7654, -1.2716]],
       
               [[ 0.2925,  0.9545, -0.6153],
                [-0.1786,  1.7654, -1.2716]],
       
               [[ 0.2925,  0.9545, -0.6153],
                [-0.1786,  1.7654, -1.2716]],
       
               [[ 0.2925,  0.9545, -0.6153],
                [-0.1786,  1.7654, -1.2716]],
       
               [[ 0.2925,  0.9545, -0.6153],
                [-0.1786,  1.7654, -1.2716]]]),
    x: Batch(
           c: tensor([[[ 0.7716,  0.7006, -1.4428,  0.9357],
                       [-0.4567, -0.1508,  1.0762,  0.4959],
                       [-1.8219, -0.5449, -1.3142,  0.1862]],
         

In [26]:
%timeit Batch.stack(batches)

78.3 µs ± 96.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.2925,  0.9545, -0.6153],
               [-0.1786,  1.7654, -1.2716],
               [ 0.2925,  0.9545, -0.6153],
               [-0.1786,  1.7654, -1.2716],
               [ 0.2925,  0.9545, -0.6153],
               [-0.1786,  1.7654, -1.2716],
               [ 0.2925,  0.9545, -0.6153],
               [-0.1786,  1.7654, -1.2716],
               [ 0.2925,  0.9545, -0.6153],
               [-0.1786,  1.7654, -1.2716],
               [ 0.2925,  0.9545, -0.6153],
               [-0.1786,  1.7654, -1.2716],
               [ 0.2925,  0.9545, -0.6153],
               [-0.1786,  1.7654, -1.2716],
               [ 0.2925,  0.9545, -0.6153],
               [-0.1786,  1.7654, -1.2716]]),
    x: Batch(
           c: tensor([[ 0.7716,  0.7006, -1.4428,  0.9357],
                      [-0.4567, -0.1508,  1.0762,  0.4959],
                      [-1.8219, -0.5449, -1.3142,  0.1862],
                      [ 0.7716,  0.7006, -1.4428,  0.9357],
                      [-0.4567, -

In [28]:
%timeit Batch.cat(batches)

141 µs ± 224 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

306 µs ± 403 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
